In [5]:
import pandas as pd
import numpy as np
import math

In [148]:
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    sum = np.sum(counts)  
    return np.sum([(-counts[i]/sum)*np.log2(counts[i]/sum) for i in range(len(elements))])

In [149]:
def InfoGain(data,split_attribute_name,target_name="class"): 
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)

    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [1]:
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):

    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    if len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    if len(features) ==0:
        return parent_node_class
        
        #Set the default value for this node --> The mode target feature value of the current node
    classes, counts = np.unique(data[target_attribute_name], return_counts=True)
    parent_node_class = classes[np.argmax(counts)]  
        
    item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
    best_feature = features[np.argmax(item_values)]
        
    tree = {best_feature:{}}
        
    features = [i for i in features if i != best_feature]
        
        
    for value in np.unique(data[best_feature]):
        value = value # might not be needed
        sub_data = data.where(data[best_feature] == value).dropna()
            
        subtree = ID3(sub_data,data,features,target_attribute_name,parent_node_class)
            
        tree[best_feature][value] = subtree
            
    return(tree) 

In [2]:
item_values = [InfoGain(data,feature,'class') for feature in features] #Return the information gain values for the features in the dataset
print(features.columns[np.argmax(item_values)])
print(features.columns[0])

NameError: name 'features' is not defined

In [6]:
data = pd.DataFrame({"color":["red","blue","red","green","red","green"],
                     "shape":["square","square","round","square","round","square"],
                     "size":["big","big","small","small","big","big"],
                     "class":["+","+","-","-","+","-"]
                     }, 
                    columns=["color","shape","size","class"])
features = data[["color","shape","size"]]
target = data["class"]
data

,color,shape,size,class
0,red,square,big,+
1,blue,square,big,+
2,red,round,small,-
3,green,square,small,-
4,red,round,big,+
5,green,square,big,-


In [158]:
tree = ID3(data,data,features.columns)

In [159]:
tree

{'color': {'blue': '+',
  'green': '-',
  'red': {'size': {'big': '+', 'small': '-'}}}}